# Part 2: Regression

Whereas with classification we use a set of features (or independent variables) to predict a discrete output (dependent variable), in regression we are trying to predict a continuous output (e.g. a real valued number).

# 1) Boston Dataset

For demonstration, we will use scikit-learn's [Boston](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) dataset. Instead of predicting discrete categories as we would in classification, with this dataset we can attempt to predict price, a continuous variable.

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

If you are going to follow along in other tutorials in the scikit-learn documentation, you will need to know the data structures used as inputs to the models. Let'see what's in the boston dataset:

In [ ]:
print(boston.keys())

The dataset is a dictionary-like object containing 5 items. The description (`DESCR`) will tell us more about the dataset:

In [ ]:
print(boston.DESCR)

So we are working on predicting the median home value from 506 observations using 13 features including crime rate, lot size, industry/commercial proportion, presence of the Charles River, nitric oxide concentration, rooms per dwelling, units built before 1940, distance to employment centers, access to highways, tax rate, school proxy, black population, and status. 

To get the variable names use the `.feature_names` attribute of the dataset, which is stored in a `numpy` array:

In [ ]:
print('Feature Names:', boston.feature_names, '\n')
print('Type of the feature_names attribute:', type(boston.feature_names), '\n')
print('Number of features:', len(boston.feature_names), '\n')

The dataset is stored in the `data` field of the dataset:

In [ ]:
print('Example first row of data:\n', boston.data[0], '\n')
print('Type of the data:', type(boston.data), '\n')
print('Shape of the features data:', boston.data.shape)

The data is stored as `numpy` array, where each row represents a data point (a house in this case), and each column represents a feature. So we have 13 variables worth of data for each of the 506 houses in this dataset.

The output variable (median price) or *y* is accessed via the `target` item in the dataset:

In [ ]:
print('Example first output value:', boston.target[0], '\n')
print('Type of output (target) data:', type(boston.target), '\n')
print('Shape of output data:', boston.target.shape)

The target array is only one dimension, lined up in order with the with the feature values in the data array.

Now that we're familiar with the input data, we need to split it up for training and testing, but first thing's first: **set the random seed!**

In [ ]:
import numpy as np
np.random.seed(10)

Now we can use the train_test_split feature:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target,
                                                    train_size=0.75, test_size=0.25)

Now we have 75% of the data as training data, and 25% of the data as testing data:

In [ ]:
print('XTrain shape:', X_train.shape, 'YTrain shape:', y_train.shape, '\n')
print('XTest shape:', X_test.shape, 'YTest shape:', y_test.shape)

In scikit-learn, as soon as you have `X_train`, `X_test`, `y_train`, and `y_test`, everything else is just a matter of choosing your mdoel and the parameters for it. But this should not be trivialized, selecting models and that model's parameters is *very* important. While we will not cover it here, choosing the correct model and parameters is the core skill of applying machine learning algorithms, and can have dramatic affects on the performance of your predictions.

# 2) Building models

The syntax in scikit-learn does not change for each model, only the parameters. It also is not very different from the classification model syntax. Examples of various models are given below:

## Linear Models

### GLM - Ordinary Least Squares Linear Regression

We'll start with a basic [OLS linear regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression):

In [ ]:
from sklearn import linear_model
lin_reg = linear_model.LinearRegression(n_jobs=1)  # CPUs to use

In [ ]:
lin_reg.fit(X_train, y_train)

We can see how well we fit the training set. When fitting classification models, the `.score` method would return mean accuracy. For regression models `.score()` returns the amount of variance in the output variable that can be explained by the model predictions. This is known as $R^2$, or R-squared. There are many other performance metrics that can be used when predicting continuous variables. See [here]() for an overview.

Let's look at the $R^2$ for the training data:

In [ ]:
print('Training data R^2: %.04f' % (lin_reg.score(X_train, y_train)))

And the test test. 

In [ ]:
print('Test data R^2: %.04f' % (lin_reg.score(X_test, y_test)))

We can plot the coefficients to see which features impact house price the most. It looks like nitric oxide concentrations (NOX) have the largest negative association and number of rooms (RM) have the largest positive association with home price. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8,5))
plt.bar(range(len(lin_reg.coef_)), lin_reg.coef_)
plt.xticks(np.arange(len(lin_reg.coef_)),boston.feature_names, fontsize=14);
plt.axhline(y=0,linestyle='--',color='k')
plt.ylabel('Regression coefficient (beta weight)', fontsize=18);
plt.xlabel('Feature Names', fontsize=18);
plt.ylim([-16.0,4]);

### GLM - Ridge Regression

If you have many features, you may want to consider regularization. 

Instead of minimizing least squares loss: 
$$ L(\beta) = \sum_i^n (y_i - \hat y_i)^2 $$ 

In ridge regression we additionally penalize the coefficients and minimize this: 

$$ L(\beta) = \sum_i^n (y_i - \hat y_i)^2  + \alpha \sum_j^p \beta^2 $$ 

Ridge regression takes a **hyerparameter**, called alpha (sometimes lambda). This hyperparameter indicates how much regularization should be done. In other words, how much to care about the coefficient penalty term vs how much to care about the sum of squared errors term. The higher the value of alpha the more regularization, and the smaller the resulting coefficients will be. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) for more.

If we use an `alpha` value of `0` then we get the same solution as the OLS regression done above. Let's prove that.

In [ ]:
from sklearn import linear_model
ridge_reg = linear_model.Ridge(alpha=0,  # regularization
                               normalize=True,  # normalize X regressors
                               solver='auto',
                               random_state = 10)  # options = ‘auto’, ‘svd’, ‘cholesky’, ‘lsqr’, ‘sparse_cg’, ‘sag'

model = ridge_reg.fit(X_train, y_train)

In [ ]:
print('Test R^2: %.04f' % (model.score(X_train, y_train)))
print('Test R^2: %.04f' % ( model.score(X_test, y_test)))

Generally we don't know what the best value hypterparameter values should be, and so we need to use some form of cross-validation to determine that value. `RidgeCV` does just that. It fits a ridge regression model by first using cross-validation to find a good value of alpha. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV) for more.

We specify the alphas we want the estimator to try. It's often a good idea to use a logarithmic space to allow for finer grained search in smaller values. Let's create the alphas list we want to use.

In [ ]:
alphas = np.logspace(-1,1,20)
alphas

In [ ]:
plt.bar(range(len(alphas)), alphas)

By default the `RidgeCV` uses "Leave One Out Cross Validation" (LOOCV). Let's fit the Ridge model

In [ ]:
ridge_cv = linear_model.RidgeCV(alphas=alphas,
                               normalize=True,
                               store_cv_values=True)
ridge_cv.fit(X_train, y_train);

Let's see how it did relative to OLS.

In [ ]:
print('Train R^2: %.04f' % (ridge_cv.score(X_train, y_train)))
print('Test R^2: %.04f' % (ridge_cv.score(X_test, y_test)))

Looks like it did a bit worse than using regular OLS! We can look at a plot showing the model performance (In mean squared error, or MSE) as a function of alpha size. Let's see

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(alphas, ridge_cv.cv_values_.mean(axis=0))
plt.xlabel('alpha (regularization hyperparameter)', fontsize=18)
plt.ylabel('CV Performance (MSE)', fontsize=18)

We can also look at the coefficients it estimated:

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(range(len(ridge_cv.coef_)), ridge_cv.coef_)
plt.xticks(np.arange(len(ridge_cv.coef_)),boston.feature_names, fontsize=12)
plt.axhline(y=0,linestyle='--',color='k')
plt.ylabel('Regression coefficient (beta weight)', fontsize=18);
plt.xlabel('Feature Names', fontsize=18);
plt.ylim([-16.0,4]);

### GLM - Elastic Net Regression

Elastic Net regression is another form of regularized regression that uses a combination of an L2 penalization (same as Ridge) and an L1 penalization (same as Lasso). See [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) for more.

In [ ]:
elastic_reg = linear_model.ElasticNet(alpha=1.0,  # penalty, 0 is OLS 
                               random_state=10,
                               selection='cyclic')  # or 'random', which converges faster

model = elastic_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

### Support Vector Regression

Support Vector Machines (SVMs) are popular and effective models that find the data points of each class that are closest to each other (the support vectors) and then find a hyperplane half way between those points. SVMs can be used in a linear fashion (as is done below) or by applying a non-linear kernel function. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) for more. 

In [ ]:
from sklearn import svm

sv_reg = svm.SVR(kernel='linear',  # ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
                 degree=3,  # only used for 'poly' above
                 gamma='auto',  # kernal coeff, default auto is 1/n_features
                 C=1.0)

model = sv_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

## Non-Linear Models

### K-nearest neighbors regression

K Nearest Neighbors uses the averaged values of the `k` data points that are closest to the predicted value in the feature space. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) for more.

In [ ]:
from sklearn import neighbors

knn_reg = neighbors.KNeighborsRegressor(n_neighbors=5,
                                        weights='uniform',  # ‘distance’ weights points by inverse of their distance
                                        algorithm='auto',  # out of ‘ball_tree’, ‘kd_tree’, ‘brute’
                                        leaf_size=30)  # for tree algorithms

model = knn_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

### Random Forest Regression

We've already used random forests for classification in the previous section, and here we'll use them for regression. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) for more.

In [ ]:
from sklearn import ensemble

rf_reg = ensemble.RandomForestRegressor(n_estimators=10,  # number of trees
                                        criterion='mse',  # how to measure fit
                                        max_depth=None,  # how deep tree nodes can go
                                        min_samples_split=2,  # samples needed to split node
                                        min_samples_leaf=1,  # samples needed for a leaf
                                        min_weight_fraction_leaf=0.0,  # weight of samples needed for a node
                                        max_features='auto',  # max feats
                                        max_leaf_nodes=None,  # max nodes
                                        n_jobs=1, # how many to run parallel
                                        random_state=10)

model = rf_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

### Boosting - AdaBoost Regression

You used an adaptive boosting, or AdaBoost, estimator to do classification in the challenge question of the previous section. Here we'll use it for regression. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for more.

In [ ]:
ab_reg = ensemble.AdaBoostRegressor(base_estimator=None,  # default is DT 
                                    n_estimators=50,  # number to try before stopping
                                    learning_rate=1.0,  # decrease influence of each additional estimator
                                    random_state=10,
                                    loss='linear')  # also ‘square’, ‘exponential’


model = ab_reg.fit(X_train, y_train)
print(model.score(X_test, y_test))

## 3) Grid Search

As with classfication, you can also use grid search on regression models.

In [ ]:
param_grid = {'n_estimators': range(10,50),
              'learning_rate': np.arange(0.01, 1, .1)}

In [ ]:
from sklearn.model_selection import GridSearchCV

model_reg = GridSearchCV(ensemble.AdaBoostRegressor(), param_grid, cv=3, iid=False)
model_reg.fit(X_train, y_train);

In [ ]:
best_index = np.argmax(model_reg.cv_results_["mean_test_score"])

print(model_reg.cv_results_["params"][best_index])
print(max(model_reg.cv_results_["mean_test_score"]))
print(model_reg.score(X_test, y_test))

## 4) Prediction

Great, not a bad fit! Let's say we come upon a house and want to guess its median value. Here are the feature values:

In [ ]:
random_house = [.00631, 17.000, 2.410, 0, .538, 6.575, 65.200, 4.090, 1.00, 296.000, 15.300, 396.900, 4.980]

for i in range(len(random_house)):
    print(boston.feature_names[i])
    print(random_house[i])
    print()

Now let's use our model to predict!

In [ ]:
model_reg.predict([random_house])

## Challenge

Choose three algorithms and use grid search to determine the best model for this dataset. Make sure to base your decision on model perfomrance on the out-of-sample test set data.